In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb  # Importando LightGBM
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import mutual_info_regression
import multiprocessing
from functools import partial
from tqdm import tqdm
import sys
import os
import warnings
import gc
import pyarrow

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

def otimizar_memoria(df, verbose=True):
    """
    Reduz o uso de memória de um DataFrame, alterando os tipos de dados das colunas.
    """
    print("\nETAPA: OTIMIZANDO O TAMANHO DO DATAFRAME")
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'TAMANHO INICIAL DO DATAFRAME: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category' and 'datetime' not in str(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'TAMANHO FINAL DO DATAFRAME: {end_mem:.2f} MB')
        print(f'PERCENTUAL DE REDUÇÃO: {100 * (start_mem - end_mem) / start_mem:.1f}%')
    return df

def _ler_um_ano(ano, caminho_base):
    """
    Função auxiliar para ler o arquivo CSV de um único ano.
    """
    caminho_arquivo = os.path.join(caminho_base, f'microdados_ed_basica_{ano}', 'dados', f'microdados_ed_basica_{ano}.csv')
    try:
        df_ano = pd.read_csv(caminho_arquivo, sep=';', encoding='latin1', low_memory=False)
    except (UnicodeDecodeError, FileNotFoundError):
        try:
            df_ano = pd.read_csv(caminho_arquivo, sep=';', encoding='utf-8', low_memory=False)
        except Exception as e_final:
            print(f"ERRO AO CARREGAR {ano}: {e_final}", file=sys.stderr)
            return None
    df_ano['NU_ANO_CENSO'] = ano
    return df_ano

def carregar_e_combinar_dados(caminho_base, intervalo_anos):
    """
    Carrega dados de múltiplos arquivos CSV em paralelo e os combina em um único DataFrame.
    """
    print("\nETAPA: CARREGANDO OS DADOS DO CSV E CRIANDO O DATAFRAME")
    print(f"INICIANDO CARGA DE DADOS DE FORMA PARALELA: ANOS {list(intervalo_anos)}")
    n_cores = multiprocessing.cpu_count()
    pool_size = max(1, n_cores - 2)

    with multiprocessing.Pool(processes=pool_size) as pool:
        func_parcial = partial(_ler_um_ano, caminho_base=caminho_base)
        dataframes = []
        with tqdm(total=len(intervalo_anos), desc="Lendo arquivos CSV em paralelo") as pbar:
            for df_ano in pool.imap_unordered(func_parcial, intervalo_anos):
                if df_ano is not None:
                    dataframes.append(df_ano)
                pbar.update()

    if not dataframes:
        print("ERRO: NENHUM DADO CARREGADO. VERIFIQUE OS CAMINHOS.", file=sys.stderr)
        sys.exit(1)

    print("\nSUBETAPA: CONCATENANDO DATAFRAMES DE ANOS")
    df_completo = pd.concat(dataframes, ignore_index=True)
    del dataframes
    gc.collect()
    print(f"SHAPE DO DATAFRAME CONCATENADO: {df_completo.shape}")

    return otimizar_memoria(df_completo)

def preprocessar_dados(df, null_threshold=0.9):
    """
    Executa uma série de etapas de pré-processamento nos dados.
    """
    print("\nETAPA: INICIANDO O PRE PROCESSAMENTO DOS DADOS")

    print("\nSUBETAPA [1]: REMOVENDO COLUNAS COM MUITOS NULOS")
    limite_nulos = len(df) * null_threshold
    cols_a_remover_nulos = [col for col in df.columns if df[col].isnull().sum() > limite_nulos]
    if cols_a_remover_nulos:
        print(f"COLUNAS COM MUITOS NULOS REMOVIDAS: {len(cols_a_remover_nulos)}")
        df.drop(columns=cols_a_remover_nulos, inplace=True)

    print("\nSUBETAPA [2]: SELECIONANDO AS COLUNAS QUE IDENTIFICAM AS ETAPAS DE ENSINO")
    sufixos_etapas = ['CRE', 'PRE', 'AI', 'AF', 'MED', 'FUND', 'TEC', 'CE', 'EJA_FUND', 'EJA_MED']
    cols_etapas = [col for col in df.columns if col.startswith('IN_') and any(sufixo in col for sufixo in sufixos_etapas)]
    print(f"COLUNAS DE ETAPAS ENCONTRADAS: {len(cols_etapas)}")

    print("\nSUBETAPA [3]: OTIMIZANDO COLUNAS INDICADORAS E PREENCHENDO NAs")
    cols_indicadoras = [col for col in df.columns if col.startswith('IN_')]
    for col in tqdm(cols_indicadoras, desc="Processando colunas indicadoras"):
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(np.int8)

    print("\nSUBETAPA [4]: CONVERTENDO CATEGORIAS EM DADOS NUMÉRICOS")
    cols_categoricas_potenciais = df.select_dtypes(include=['object', 'category']).columns.tolist()
    cols_categoricas_potenciais += [col for col in df.columns if col.startswith('TP_')]
    cols_a_ignorar_cat = ['CO_ENTIDADE', 'NO_ENTIDADE', 'DS_ENDERECO']
    cols_a_converter = [c for c in set(cols_categoricas_potenciais) if c in df.columns and c not in cols_a_ignorar_cat]

    print(f"COLUNAS CATEGORICAS A CONVERTER: {len(cols_a_converter)}")
    for col in tqdm(cols_a_converter, desc="Convertendo colunas categóricas"):
        df[col] = pd.Categorical(df[col]).codes

    print("\nSUBETAPA [5]: REMOVENDO OUTLIERS USANDO CAPPING EM PERCENTIL 99")
    cols_quantitativas = [c for c in df.select_dtypes(include=np.number).columns if c.startswith('QT_')]
    if cols_quantitativas:
        print(f"COLUNAS QUANTITATIVAS A SEREM TRATADAS: {len(cols_quantitativas)}")
        for col in tqdm(cols_quantitativas, desc="Tratando outliers"):
            p99 = df[col].quantile(0.99)
            if p99 > 0:
                df.loc[df[col] > p99, col] = p99

    print("\nSUBETAPA [6]: PREENCHENDO NAN COM MEDIANA")
    for col in tqdm(df.select_dtypes(include=np.number).columns, desc="Preenchendo NaNs com mediana"):
        if df[col].isnull().any():
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)

    print("\nSUBETAPA [7]: REMOVENDO COLUNAS SEM VARIÂNCIA")
    cols_a_manter_sempre = ['CO_ENTIDADE', 'NU_ANO_CENSO'] + cols_etapas
    cols_verificar_variancia = [c for c in df.select_dtypes(include=np.number).columns if c not in cols_a_manter_sempre]

    cols_a_remover_variancia = [col for col in cols_verificar_variancia if df[col].nunique(dropna=False) <= 1]
    if cols_a_remover_variancia:
        print(f"COLUNAS IDENTIFICADAS SEM VARIÂNCIA: {len(cols_a_remover_variancia)}")
        df.drop(columns=cols_a_remover_variancia, inplace=True)

    print(f"PRE-PROCESSAMENTO CONCLUIDO. SHAPE FINAL: {df.shape}")

    print("\nSUBETAPA: OTIMIZANDO DATAFRAME PRE PROCESSADO")
    df = otimizar_memoria(df, verbose=False)
    return df, cols_etapas

def _processar_chunk_temporal(df_chunk):
    """
    Processa um chunk de DataFrame para criar features temporais (lag e growth rate).
    """
    df_chunk = df_chunk.sort_values(['CO_ENTIDADE', 'NU_ANO_CENSO'])
    cols_para_lag = [col for col in df_chunk.columns if col.startswith(('QT_MAT_', 'QT_DOC_', 'QT_TUR_'))]
    grouped = df_chunk.groupby('CO_ENTIDADE')
    for col in cols_para_lag:
        df_chunk[f'{col}_lag1'] = grouped[col].shift(1)
        df_chunk[f'{col}_growth_rate_lag1'] = grouped[col].pct_change(1).shift(1)
    return df_chunk

def _generate_df_chunks(df, chunks_de_codigos):
    """
    Gerador que produz chunks do DataFrame baseados em listas de códigos de entidade.
    """
    for chunk in chunks_de_codigos:
        yield df[df['CO_ENTIDADE'].isin(chunk)].copy()

def criar_features_temporais(df):
    """
    Cria features temporais (lags) de forma paralela.
    """
    print("\nETAPA: CRIANDO FEATURES TEMPORAIS (LAGS) EM PARALELO")
    print("SUB-ETAPA: CALCULANDO PARAMETROS (NÚCLEOS, CHUNKS NECESSÁRIOS, ESCOLAS)")
    n_cores = multiprocessing.cpu_count()
    pool_size = max(1, n_cores - 2)

    codigos_escolas = df['CO_ENTIDADE'].unique()
    num_chunks = min(pool_size * 4, len(codigos_escolas))

    chunks_de_codigos = np.array_split(codigos_escolas, num_chunks)
    df_chunk_generator = _generate_df_chunks(df, chunks_de_codigos)
    print(f"NÚMERO DE CHUNKS: {num_chunks}")

    resultados_chunks = []
    print("SUB-ETAPA: COMEÇANDO O PARALELISMO DO PROCESSAMENTO DAS CHUNKS")
    with multiprocessing.Pool(processes=pool_size) as pool:
        with tqdm(total=num_chunks, desc="Processando chunks de escolas") as pbar:
            for resultado in pool.imap_unordered(_processar_chunk_temporal, df_chunk_generator):
                resultados_chunks.append(resultado)
                pbar.update()

    print("SUB-ETAPA: CONCATENANDO OS CHUNKS PROCESSADOS")
    df_final = pd.concat(resultados_chunks, ignore_index=True)
    del resultados_chunks
    gc.collect()
    print(f"SHAPE DA CONCATENAÇÃO COM LAGS: {df_final.shape}")

    print("SUB-ETAPA: FILTRANDO LINHAS COM LAGS INVÁLIDOS (NaN, Inf)")
    cols_para_lag = [col for col in df.columns if col.startswith(('QT_MAT_', 'QT_DOC_', 'QT_TUR_'))]
    subset_dropna = [f'{c}_lag1' for c in cols_para_lag if f'{c}_lag1' in df_final.columns]
    df_final.dropna(subset=subset_dropna, inplace=True)
    df_final.replace([np.inf, -np.inf], np.nan, inplace=True)

    print("SUB-ETAPA: PREENCHENDO COLUNAS NUMÉRICAS NA COM ZERO")
    numeric_cols = df_final.select_dtypes(include=np.number).columns
    df_final[numeric_cols] = df_final[numeric_cols].fillna(0)

    print(f"SHAPE FINAL APÓS CRIAÇÃO DE FEATURES: {df_final.shape}")
    return df_final

def _worker_modelagem_etapa(col_etapa, file_path, top_n_features, todas_as_etapas):
    """
    Worker para o paralelismo: processa uma única etapa de ensino, desde a seleção
    de features até a modelagem e avaliação.
    """
    print(f"\nSUB-ETAPA: INICIANDO ETAPA DE ENSINO EM PARALELISMO: [{col_etapa}]")

    df_full = pd.read_feather(file_path)
    dados_etapa = df_full[df_full[col_etapa] == 1].copy()
    del df_full
    gc.collect()

    if len(dados_etapa) < 50:
        print(f"[{col_etapa}] PULANDO: DADOS INSUFICIENTES ({len(dados_etapa)} amostras).")
        return []

    sufixo_alvo = col_etapa.replace('IN_', '')
    alvo = f'QT_MAT_{sufixo_alvo}'

    if alvo not in dados_etapa.columns or dados_etapa[alvo].nunique() < 2:
        print(f"[{col_etapa}] PULANDO: VARIÁVEL ALVO '{alvo}' NÃO ENCONTRADA OU SEM VARIÂNCIA.")
        return []

    features_candidatas = [
        c for c in dados_etapa.select_dtypes(include=np.number).columns
        if c not in [alvo, 'CO_ENTIDADE', 'NU_ANO_CENSO'] and
           c not in todas_as_etapas and
           not (c.startswith('QT_MAT_') and '_lag' not in c)
    ]
    features_candidatas = [c for c in features_candidatas if dados_etapa[c].nunique() > 1]

    if not features_candidatas:
        print(f"[{col_etapa}] PULANDO: NENHUMA FEATURE CANDIDATA VÁLIDA.")
        return []

    X_cand = dados_etapa[features_candidatas]
    y_cand = np.log1p(dados_etapa[alvo])

    try:
        mi_scores = mutual_info_regression(X_cand, y_cand, random_state=42)
        df_mi = pd.DataFrame({'feature': features_candidatas, 'mi_score': mi_scores})
        features_selecionadas = df_mi[df_mi['mi_score'] > 0].sort_values('mi_score', ascending=False).head(top_n_features)['feature'].tolist()
    except Exception as e:
        print(f"[{col_etapa}] ERRO NO CALCULO DA MUTUAL INFORMATION, PULANDO COLUNA: {e}", file=sys.stderr)
        features_selecionadas = [f for f in features_candidatas if f.endswith("_lag1")][:20]

    if not features_selecionadas:
        print(f"[{col_etapa}] NENHUMA FEATURE FOI SELECIONADA APÓS MI. PULANDO.")
        return []

    X = dados_etapa[features_selecionadas]
    y = np.log1p(dados_etapa[alvo])
    X, y = X.align(y, axis=0, join='inner')

    if len(X) < 20:
        print(f"[{col_etapa}] PULANDO: POUCAS AMOSTRAS ({len(X)}) APÓS ALINHAMENTO.")
        return []

    # Modelos a serem treinados e avaliados
    modelos = {
        "Regressão Linear": Pipeline([('scaler', RobustScaler()), ('modelo', LinearRegression())]),
        "LightGBM": lgb.LGBMRegressor(random_state=42, n_jobs=1, verbosity=-1),
        "Random Forest": RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=1, max_depth=10, min_samples_leaf=5)
    }

    # Métricas a serem calculadas
    scoring_metrics = {
        'r2': 'r2',
        'mae': 'neg_mean_absolute_error',
        'mse': 'neg_mean_squared_error',
        'rmse': 'neg_root_mean_squared_error',
        'mape': 'neg_mean_absolute_percentage_error'
    }

    resultados_etapa = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for nome_modelo, modelo in modelos.items():
        try:
            scores = cross_validate(modelo, X, y, cv=kf, scoring=scoring_metrics, n_jobs=1)
            
            # Médias das métricas
            r2_mean = scores['test_r2'].mean()
            mae_mean = -scores['test_mae'].mean()
            mse_mean = -scores['test_mse'].mean()
            rmse_mean = -scores['test_rmse'].mean()
            mape_mean = -scores['test_mape'].mean()

            # Desvio padrão das métricas
            r2_std = scores['test_r2'].std()
            mae_std = scores['test_mae'].std()
            rmse_std = scores['test_rmse'].std()
            
            # Cálculo do R² Ajustado
            n_samples = len(X)
            n_features = len(features_selecionadas)
            adj_r2_mean = 1 - (1 - r2_mean) * (n_samples - 1) / (n_samples - n_features - 1)

            resultados_etapa.append({
                'contexto': col_etapa,
                'modelo': nome_modelo,
                'R2_Media_CV': r2_mean,
                'R2_Ajustado_Media_CV': adj_r2_mean,
                'MAE_Media_CV': mae_mean,
                'MSE_Media_CV': mse_mean,
                'RMSE_Media_CV': rmse_mean,
                'MAPE_Media_CV': mape_mean,
                'R2_Std_CV': r2_std,
                'MAE_Std_CV': mae_std,
                'RMSE_Std_CV': rmse_std,
                'num_amostras': n_samples,
                'num_features': n_features,
                'features_utilizadas': ", ".join(features_selecionadas)
            })
        except Exception as e:
            print(f"[{col_etapa}] ERRO AO TREINAR O MODELO: {nome_modelo} POIS: {e}", file=sys.stderr)
            continue

    del dados_etapa, X, y, X_cand, y_cand
    gc.collect()

    print(f"[{col_etapa}] CONCLUÍDO!")
    return resultados_etapa

def analisar_e_modelar_paralelo(df, cols_etapas, top_n_features):
    """
    Orquestra a modelagem em paralelo, distribuindo as etapas de ensino pelos workers.
    """
    print("\nETAPA: MODELAGEM DE MODELOS COM BASE NAS FEATURES MAIS CORRELACIONADAS")

    n_cores = multiprocessing.cpu_count()
    pool_size = max(1, n_cores - 2)
    temp_file_path = "temp_data_for_workers.feather"

    try:
        print(f"NÚCLEOS DA CPU EM USO: {pool_size}")
        print("SUB-ETAPA: SALVANDO DATAFRAME COMPLETO EM ARQUIVO TEMPORÁRIO (Feather)")
        df.reset_index(drop=True).to_feather(temp_file_path)
        del df
        gc.collect()

        func_parcial = partial(
            _worker_modelagem_etapa,
            file_path=temp_file_path,
            top_n_features=top_n_features,
            todas_as_etapas=cols_etapas
        )

        resultados_finais = []
        print("SUB-ETAPA: COMEÇANDO O PARALELISMO DO PROCESSAMENTO DAS MODELAGENS POR ETAPAS")
        with multiprocessing.Pool(processes=pool_size) as pool:
            with tqdm(total=len(cols_etapas), desc="Processando Etapas de Ensino") as pbar:
                for resultado_etapa in pool.imap_unordered(func_parcial, cols_etapas):
                    if resultado_etapa:
                        resultados_finais.extend(resultado_etapa)
                    pbar.update()

        print("\nANALISE E MODELAGEM CONCLUIDAS.")
        return resultados_finais

    finally:
        print("SUB-ETAPA: REMOVENDO O ARQUIVO TEMPORÁRIO")
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

def resumir_e_exportar(resultados, caminho_saida):
    """
    Resume os resultados da modelagem, imprime na tela e exporta para um arquivo CSV.
    """
    print("\nETAPA: RESUMO DOS RESULTADOS")
    if not resultados:
        print("\nNENHUM RESULTADO DE ML GERADO.")
        return

    df_resultados = pd.DataFrame(resultados)

    resultados_razoaveis = df_resultados[df_resultados['R2_Media_CV'] > 0].copy()
    if resultados_razoaveis.empty:
        print("\nNENHUM MODELO COM R² POSITIVO FOI GERADO. VERIFIQUE OS DADOS E FEATURES.")
        return
        
    print(f"SUB-ETAPA: EXPERIMENTOS VÁLIDOS (R² > 0): {len(resultados_razoaveis)}")

    resultados_razoaveis.rename(columns={'contexto': 'etapa'}, inplace=True)
    
    metricas_agg = {
        'R2_Media_CV': ['mean', 'median', 'std'],
        'R2_Ajustado_Media_CV': ['mean', 'median'],
        'RMSE_Media_CV': ['mean', 'median'],
        'MAE_Media_CV': ['mean', 'median'],
        'num_amostras': ['sum']
    }
    
    sumario = resultados_razoaveis.groupby('modelo').agg(metricas_agg).sort_values(('R2_Media_CV', 'mean'), ascending=False)
    
    print("\n" + "="*50)
    print("DESEMPENHO AGREGADO POR MODELO (MÉDIAS)")
    print("="*50)
    print(sumario.round(3))

    print("\n" + "="*70)
    print("DESEMPENHO DETALHADO POR ETAPA E MODELO (ORDENADO POR ETAPA E MELHOR R²)")
    print("="*70)
    
    cols_display = ['R2_Media_CV', 'R2_Ajustado_Media_CV', 'RMSE_Media_CV', 'MAE_Media_CV', 'MAPE_Media_CV', 'num_amostras']
    sumario_por_etapa = resultados_razoaveis.groupby(['etapa', 'modelo'])[cols_display].mean().sort_values(['etapa', 'R2_Media_CV'], ascending=[True, False])
    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(sumario_por_etapa.round(3))

    try:
        diretorio_saida = os.path.dirname(caminho_saida)
        if diretorio_saida and not os.path.exists(diretorio_saida):
            os.makedirs(diretorio_saida)
        df_resultados.to_csv(caminho_saida, index=False, encoding='utf-8-sig')
        print(f"\nRESULTADOS COMPLETOS EXPORTADOS PARA: {caminho_saida}")
    except Exception as e:
        print(f"ERRO AO SALVAR CSV: {e}", file=sys.stderr)

def main():
    """
    Função principal que orquestra todo o pipeline de execução.
    """
    print("ETAPA INICIAL: INICIANDO SCRIPT DE MODELAGEM")

    # --- PARÂMETROS DE EXECUÇÃO ---
    AMOSTRAGEM_ATIVA = True
    FRACAO_AMOSTRA = 0.1
    CAMINHO_BASE = '/home/vmuser131/gdrive/MESTRADO_CC/MEU_PROJETO/MICRODADOS_CENSO'
    CAMINHO_SAIDA_CSV = '/home/vmuser131/gdrive/MESTRADO_CC/MEU_PROJETO/resumo_desempenho_ml_detalhado.csv'
    ANOS = range(2011, 2022)
    TOP_N_FEATURES_POR_MODELO = 25
    # -----------------------------

    df_completo = carregar_e_combinar_dados(CAMINHO_BASE, ANOS)

    if AMOSTRAGEM_ATIVA:
        print("\nETAPA: INICIANDO AMOSTRAGEM DE ESCOLAS")
        print(f"SHAPE INICIAL ANTES DA AMOSTRA: {df_completo.shape}")
        codigos_escolas = df_completo['CO_ENTIDADE'].unique()
        tamanho_amostra = int(len(codigos_escolas) * FRACAO_AMOSTRA)
        print(f"TOTAL DE ESCOLAS ÚNICAS: {len(codigos_escolas)}")
        print(f"ESCOLAS A SEREM AMOSTRADAS ({FRACAO_AMOSTRA*100}%): {tamanho_amostra}")

        np.random.seed(42)
        escolas_amostra = np.random.choice(codigos_escolas, size=tamanho_amostra, replace=False)
        df_principal = df_completo[df_completo['CO_ENTIDADE'].isin(escolas_amostra)].copy()
        
        print(f"SHAPE APÓS AMOSTRA: {df_principal.shape}")
        del df_completo
        gc.collect()
    else:
        df_principal = df_completo

    df_processado, cols_etapas_encontradas = preprocessar_dados(df_principal)
    del df_principal
    gc.collect()

    df_com_features = criar_features_temporais(df_processado)
    del df_processado
    gc.collect()

    resultados_ml = analisar_e_modelar_paralelo(
        df=df_com_features,
        cols_etapas=cols_etapas_encontradas,
        top_n_features=TOP_N_FEATURES_POR_MODELO
    )

    resumir_e_exportar(resultados_ml, CAMINHO_SAIDA_CSV)
    print("\nSCRIPT CONCLUIDO!")

if __name__ == "__main__":
    main()


ETAPA INICIAL: INICIANDO SCRIPT DE MODELAGEM

ETAPA: CARREGANDO OS DADOS DO CSV E CRIANDO O DATAFRAME
INICIANDO CARGA DE DADOS DE FORMA PARALELA: ANOS [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


Lendo arquivos CSV em paralelo: 100%|██████████| 11/11 [02:21<00:00, 12.83s/it]



SUBETAPA: CONCATENANDO DATAFRAMES DE ANOS
SHAPE DO DATAFRAME CONCATENADO: (2592108, 370)

ETAPA: OTIMIZANDO O TAMANHO DO DATAFRAME
TAMANHO INICIAL DO DATAFRAME: 7317.20 MB
TAMANHO FINAL DO DATAFRAME: 1673.56 MB
PERCENTUAL DE REDUÇÃO: 77.1%

ETAPA: INICIANDO AMOSTRAGEM DE ESCOLAS
SHAPE INICIAL ANTES DA AMOSTRA: (2592108, 370)
TOTAL DE ESCOLAS ÚNICAS: 280489
ESCOLAS A SEREM AMOSTRADAS (10.0%): 28048
SHAPE APÓS AMOSTRA: (258843, 370)

ETAPA: INICIANDO O PRE PROCESSAMENTO DOS DADOS

SUBETAPA [1]: REMOVENDO COLUNAS COM MUITOS NULOS

SUBETAPA [2]: SELECIONANDO AS COLUNAS QUE IDENTIFICAM AS ETAPAS DE ENSINO
COLUNAS DE ETAPAS ENCONTRADAS: 45

SUBETAPA [3]: OTIMIZANDO COLUNAS INDICADORAS E PREENCHENDO NAs


Processando colunas indicadoras: 100%|██████████| 210/210 [00:00<00:00, 1643.74it/s]


SUBETAPA [4]: CONVERTENDO CATEGORIAS EM DADOS NUMÉRICOS


...

[IN_FUND_AI] CONCLUÍDO!


Processando Etapas de Ensino:  98%|█████████▊| 44/45 [13:47<02:00, 120.27s/it]

[IN_FUND] CONCLUÍDO!


Processando Etapas de Ensino: 100%|██████████| 45/45 [14:50<00:00, 19.79s/it] 



ANALISE E MODELAGEM CONCLUIDAS.
SUB-ETAPA: REMOVENDO O ARQUIVO TEMPORÁRIO

ETAPA: RESUMO DOS RESULTADOS
SUB-ETAPA: EXPERIMENTOS VÁLIDOS (R² > 0): 30

DESEMPENHO AGREGADO POR MODELO (MÉDIAS)
                 R2_Media_CV              R2_Ajustado_Media_CV         \
                        mean median   std                 mean median   
modelo                                                                  
LightGBM               0.913  0.924  0.07                0.913  0.924   
Random Forest          0.908  0.920  0.07                0.908  0.920   
Regressão Linear       0.776  0.770  0.03                0.776  0.770   

                 RMSE_Media_CV        MAE_Media_CV        num_amostras  
                          mean median         mean median          sum  
modelo                                                                  
LightGBM                 0.257  0.266        0.165  0.159       550457  
Random Forest            0.267  0.270        0.170  0.166       550457  
Regre